<img style="float: right;"  src="images/LogoP.jpg" width="200">

# Nucleo F303RE Zero Hardware Board

# Board 01 : The F303RE Board

This document describes the first steps to work with the SLab system when using the **STM32 NucleoF3403RE** hardware board. That includes programming the hardware board firmware.

In order to use this **notebook** you need to have a **STM32 Nucleo F3403RE** demonstration board.

Version 1.2 (18/5/2019)  

License information is at the end of the document

---
**Bill Of Materials (BOM):**

* STM32 NucleoF3403RE demonstration board
* USB Mini Communication cable

---

## Introduction

The **hardware board** is the basic instrument in the SLab Python project. It shall include, at least, two **DACs** and four **ADCs**. A **Digital to Analog Converter** (DAC) is a device that, given an input digital number, can generate a voltage output proportional to this number. An **Analog to Digital Converter** (ADC) is a device that, given a voltage in its input, measures it and returns a digital number that is proportional to the measured voltage. So, if we have a circuit to test, we can force two voltages using the two DACs and we can measure up to four node voltages using the four ADCs.

The board communicates with a PC using a **serial COM link**. Classic serial RS-232 links are not implemented in most modern PCs, so the serial connection is usually implemented inside an **USB** link. The board is usually powered by the PC so no external power supplies are required.

Any programmable microcontroller board that features a COM link, and, at least, two DACs and four ADCs, can be used as the hardware board if it is programmed with the proper firmware. An Arduino UNO board won't do, for instance, because although it features several ADCs, it doesn't include any DAC.

The board firmware basically waits for a command request from the PC. When a command is received it interacts with the ADCs and DACs depending on the request. On some requests, information is brought back to the PC when the command ends.

In this document we will use the [STM32 Nucleo64 F303RE](http://www.st.com/content/st_com/en/products/evaluation-tools/product-evaluation-tools/mcu-eval-tools/stm32-mcu-eval-tools/stm32-mcu-nucleo/nucleo-f303re.html) demonstration board, shown in the figure below, as the hardware board. As the full name is too long, we will call it the **F303RE Board** in the rest of this document. 

![NucleoF303RE Board](images\Board_01\F303RE.jpg)

This is an **official** SLab hardware board in the sense that a firmware is provided for it. **SLab** does not have any relationship with **ST**. We just use their board and provide a firmware. Consult the [SLab Board Folder](http://localhost:8888/notebooks/Boards) for up to date information of any other officially supported boards.  
As the communicatuion protocol is documented, you are free to use any non official board but, in this case, writing the firmware is up to you.

The [F303RE Board](http://www.st.com/content/st_com/en/products/evaluation-tools/product-evaluation-tools/mcu-eval-tools/stm32-mcu-eval-tools/stm32-mcu-nucleo/nucleo-f303re.html) is a microcontroller board that [ST](http://www.st.com/) provides as a way to demonstrate one of their ARM Cortex 32 bit microcontrollers, the [STM32F303RE](http://www.st.com/content/st_com/en/products/microcontrollers/stm32-32-bit-arm-cortex-mcus/stm32f3-series/stm32f303/stm32f303re.html). This microcontroller is quite interesting because is part of the [STM32F3](https://www.st.com/en/microcontrollers-microprocessors/stm32f3-series.html?querycriteria=productId=SS1576) fammily. Microcontrollers on this familly feature a 32-bit ARM Cortex M4 core (with FPU and DSP instructions) running at 72 MHz. The core is surrounded with a lot of peripherals some of them specifically designed for analog processing. The interesting peripherals respect to the **SLab** systems are:

* Up to four independent 12bit ADCs with 5 MSPS (Million Samples Per Second) capability
* Up to three 12bit DACs (some with analog drivers)
* Up to four embedded Operational Amplifiers

The ST manufacturer produces the Nucleo Board and sells them practically at cost price to promote their microcontrollers. ST produces, also, the Discovery series of boards that usually include a microcontroller and some peripherals. There are other families of boards produced by ST, but the Nucleo and Discovery series are the cheapest ones. 

There are three series of Nucleo boards, the Nucleo 32 , Nucleo 64 and Nucleo 144 that feature 32 pin, 64 in or 144 pin microcontrollers. The **F303RE board** is one the the intermediate 64 pin ones. All the Nucleo boards are [MBED](https://www.mbed.com/en/) enabled. That means that they all are programmable using a web based MBED compiler at its [developer site](https://developer.mbed.org/). So you don't need to install any toolchain to program a Nucleo board. In fact, the first versions of the SLab firmware were compiled using MBED. The current version of the **F303RE Board** firmware has been developed on [ChibiOS/RT](http://www.chibios.org/dokuwiki/doku.php), however.

## Some board details

The **Hardware Board** is the board that interfaces both with the host PC and the circuit we want to measure. This board need to have, at least, one USB connector with a **serial over USB** link for communication with the host PC, two [DAC](https://en.wikipedia.org/wiki/Digital-to-analog_converter) outputs and four [ADC](https://en.wikipedia.org/wiki/Analog-to-digital_converter) inputs.

Any boad that satify the previous requirements can be used as a **SLab Hardware Board** as long as it includes the needed **Firmware** to communicate with the PC. Although there are plans to suppor other boards, currently the best supported board is the [STM32F303RE Nucleo64 board](https://www.st.com/en/evaluation-tools/nucleo-f303re.html). This is the board provided as part of the **SLab Kit**. You can see an image of the board below.

![Board Elements](images/Board_01/board_elements.jpg)

Now we will describe the most important elements on the board, respect to the SLab usage. If at some point you don't understand the explanations, don't worry, you don't need to know all about the board internals to use it. First thing is the connection to the **host PC**:

* A **mini USB** connector, inside the pale blue box on the image, to connect to the host PC.

The board features **LEDs**, shown iside the three red boxes:

* A communication bicolor red/green **LD1** LED on the top right.

* An user green **LD2** LED on the center right.

* A power red **LD3** LED on the center left.

The communication **LD1** LED is a bicolor led that indicates the state of the debug link with the host PC. In normal operation it will light in **red** color. During the firmware upload operation it will link. It will only shine in solid **green** color after uploading a new firmware and until the next power-up.

The **LD2** LED is the user LED. Unfortunatelly the user LED is connected to the same microcontroller line of the **second DAC** output. That limits the **DAC** performance. From an user point of view, if you see light on this LED, that means that the **DAC2** output quatlity is starting to degrade. There is a documented method to remove this LED connection and access the full performance of **DAC2** but it requires some SMD desoldering.

When you receive the **SLab Kit** it is possible for the board to come with a yellow sticker that contains a number or a number followed by an **"X"** letter. If your board contains this sticker and it includes the **"X"** letter, it means that the **DAC2** has been disconnected from the **LD2** LED as described in [this document](http://localhost:8888/notebooks/Boards/Nucleo_F303RE_Zero/Board%2003%20-%20DAC2%20Improvement.ipynb). This means that **DAC2** will have improved operation but it also means that the user **LD2** cannot be used.

The red **LD3** LED is the power LED. In indicates when the main CPU is powered. It should lit whenever the board is connected to a host PC. The board supports power negotiation with the PC. If the PC is not able to provide the power the board needs, this LED will turn off and the **SLab System** will be non operational.  
It is interesting to note that the power **LD3** LED is connected to the internal $5V$ board supply rail that comes from the USB cable. This supply goes through a $3.3V$ LD39050 regulator before reaching the main MCU power rails. If you shortcircuit the breadboard power rails, no harm will be made to the board or the USB connection and, in fact, the **LD3** will continue to light-up as the $5V$ rail is unaffected when the $3.3V$ rail is shorted.

There are two big two column male **ST morpho** connectors, shown inside the yellow rectangles, on the sides of the board:

* Connector **CN7** on the left side

* Connector **CN10** on the right side

Those connectors give access to all the MCU pins and are used to interface with the circuit we will measure.

There are two buttons in the board, shown inside the green rectangles:

* Blue **User** button that implements the **Halt** function

* Black **Reset** button to rese the board

The user button implements the firmware **Halt** function, when you press this button the firmware stopts watever it is doing, returns an error code, and waits for new commands. This function is interesting if you, by mistake, send a too long to complete command to the board.

The **reset** button gives access to the main MCU reset line. If you press this button the MCU will reset and you will need to reconnect to the board. This button is useffull if the board hangs-up and is not recoverable by the **Halt** function.

To end this board description we have three jumper locations on the board, shown inside the orange boxes in the image. From top to bottom of the board:

* **CN2** ST-Link pair of jumpers

* **JP5** power selection jumper

* **JP6** $I_{DD}$ measurement jumper

Jumpers are important because they are detachable from the board and, if you miss them, the board won't operate as intended.

The two top jumpers at **CN2** connect the main MCU debug port with the USB connector. You don't need those jumpers for normal operation of the board, but, if you remove them, you won't be able to upload any new firmware on the board.

The **JP5** jumper is a two position jumper that selects how the main MCU is powered. In order to power the MCU using the USB connector, it shall be on the right *U5V* position. If you put it on the left *E5V* position, you will need to power the MCU using the **E5V** pin at the **CN7** connector. If you take out the jimpler, the MCU won't be powered at all.

The final **JP6** jumper connects the $3.3V$ rail with the main MCU $V_{DD}$ input. This jumper is meant to be substituted by an amperimeter to measure the current consumed by the MCU. You should never take off this jumper as it will remove the MCU power input.

## Board Support

The **F303RE** board has available male output pins both in the top and bottom sides. A problem with bottom side pins is that they can short if they touch any metallic object on the table.

The first thing you should do before using the board is to add three spacers below the board and bolt them to three M3 screws as shown in the figure.

![Board Screws](images/Board_01/screws.jpg)

It's better to use plastic spacers because they don't conduct current. If you use metallic spacers be sure that they don't touch any electrically active node of the board.
Note that screw **number 3** is different because it needs to be trimmed. There is too little clearance to the Arduino **CN5** connector on this holes that a normal screw won't fit.

## Board Power

This section provides some more details about the board power management. This section is not crytical so, if you don't want to know about this subject, you can skip it.

The following image shows the schematic of the board power management. 

![Power Schematic](images/Board_01/power_sch.png)

If you are curious, you can also read the rest of the board [schematics](http://localhost:8888/files/Boards/Nucleo_F303RE_Zero/PDF/STM32F303RE%20Nucleo%20Schematics.pdf).

Some of the nodes have special meanings:

* **U5V** is the $5V$ rail obtained from the PC USB connection
* **VIN** is a pin available on **CN7** that enables powering the MCU from an external unregulated supply
* **E5V** is a pin available on **CN8** that enables powering the MCU from an external regulated $5V$ supply
* **+5V** is the the power rail, **E5V** or **U5V**, selected by **JP5** to power the main MCU
* **+3V3** is the $3.3V$ rail before the **JP6** current measurement jumper
* **VDD** is the $3.3V$ rail after the **JP6** current measurement jumper

Note that transistor **T2** connects the **U5V** rail to the **JP5** jumper. This transistor will only turn when the **PWR_ENn** signal is enabled after the power negotiation with the PC succeeds. If your PC is not able to provide the required power, you can use an external power supply. In this case you would put the **JP5** jumper on the left *E5V* position and will use either the **E5V** or the **VIN** pins to power the main MCU. This option has the added benefit that, as the MCU is not powered by the host PC, the PC is somewhat more proteccted against user errors.  
The [Nucleo64 Board Manual](http://localhost:8888/files/Boards/Nucleo_F303RE_Zero/PDF/STRM32%20Nucleo64%20User%20Manual.pdf) in the **Power supply and power selection** section gives a more in depth explanation about those options.

As you can see in the schematic the MCU **VDD** $3.3V$ rail is provided by the **U4** regulator. That isolates somewhat the MCU supply from the USB power rail. That offers some protection if, for instance, you short the the $V_{DD}$ rails on the breadboard when doing some experiments.

See also that the red **LD3** LED is connected to the $5V$ rail so, as explained before, having it lit does not mean that the $3V3$ rail is available.

## Board Documentation

The **STM32F303RE Nucleo64** board is a quite capable board. Kudos to ST for providing such a small, powerfull, versatile and well documented board.

In the folder:

$\qquad$ `SLab/Jupyter/Boards/Nucleo_F303RE_Zero/PDF`

there are some documents about this board, or some of its main elements:

* Nucleo64 board [User Manual](http://localhost:8888/files/Boards/Nucleo_F303RE_Zero/PDF/STRM32%20Nucleo64%20User%20Manual.pdf)

* Board [Schematic](http://localhost:8888/files/Boards/Nucleo_F303RE_Zero/PDF/STM32F303RE%20Nucleo%20Schematics.pdf)

* Main **STM32F303RE** MCU [datasheet](http://localhost:8888/files/Boards/Nucleo_F303RE_Zero/PDF/STM32F303%20Datasheet.pdf)

* STM32F303 **mixed signal family** MCU [reference](http://localhost:8888/files/Boards/Nucleo_F303RE_Zero/PDF/STM32F303%20Reference.pdf)

All those documents are property of [ST](https://www.st.com/content/st_com/en.html) and are the main documents used to develop the board **firmware**.   
You can obtain more up to date documentation about the **STM32F303RE Nucleo64** board in its [web page](https://www.st.com/en/evaluation-tools/nucleo-f303re.html).

The current firmware for this board has been developed using the [ChibiOS/RT](http://www.chibios.org/dokuwiki/doku.php) operating system in a custom [Eclipse](https://www.eclipse.org/) setup.  
Code compilation uses the [GNU-ARM compiler toolchain](https://developer.arm.com/tools-and-software/open-source-software/gnu-toolchain/gnu-rm).

The board is compatible with the [MBED](https://www.mbed.com/en/) system. Board details for **MBED** operation can be found on this [link](https://os.mbed.com/platforms/ST-Nucleo-F303RE/). In fact, the first versions of the **SLab** firmware were developed on **MBED**.  

![MBED](images/Board_01/mbed_small.png)

If at some point of time you are bored of **SLab**, you can use the **MBED Cloud Compliler** to learn about **firmware programming** using the same Nucleo64 board. If, at some time, you want to return to **SLab**, you just need to reupload the **SLab firmware**.

## Connecting the board to the PC

In order to use the board, in our case, the F303RE Board, we need to connect it to a PC. In most cases it will be automatically detected and drivers will be loaded for if needed you can manually install the drivers before connecting the board.

The Nucleo boards use a ST-LINK/V2-1 interface to communicate with the PC. The Windows drivers can be located inside the [F303RE Board page](http://www.st.com/content/st_com/en/products/evaluation-tools/product-evaluation-tools/mcu-eval-tools/stm32-mcu-eval-tools/stm32-mcu-nucleo/nucleo-f303re.html). If you want, you can also obtain some installation information on the [MBED page](https://developer.mbed.org/platforms/ST-Nucleo-F303RE/) of the F303RE Board.

The USB protocol enables a connected system to be seen as several different devices at the same time. If all goes well, after connecting the board to the PC, it will be seen with three different interfaces:

* A mass storage device (like an external hard disk)
* A serial communication COM port
* A debug port

We will use the storage device just once to transfer the firmware to the board. Most of the time we will use the COM port as this is the link for the requests sent from the Python code to the hardware board. We won't never use the debug communication link. Neither is needed for normal web based MBED programming.

Most of the SLab work has been developed in a **Windows machine**, so no intensive checks have been performed on **Linux** and no test at all has been developed on **MAC** computers.

It has been tested that the drivers install automatically, at least for the mass storage and COM port on **Ubuntu Linux**. 

The Nucleo board includes two microcontrollers, one, a **STM32F103** next to the USB connector, implements the ST-LINK/V2-1 that communicates with the PC. And the second one, the bigger **STM32F303RE** 64 pin chip at the center of the board, is where we store our firmware. So, in fact, we have two firmwares in the board, one for each microcontroller.

ST releases from time to time new firmwares for the STM32F103 microcontroller that takes care of the ST-LINK connection. You can find an installer program for this firmware on the [F303RE board page](http://www.st.com/content/st_com/en/products/evaluation-tools/product-evaluation-tools/mcu-eval-tools/stm32-mcu-eval-tools/stm32-mcu-nucleo/nucleo-f303re.html). It is recommended to run this installer to guarantee that the STLINK firmware is up to date, but it never hurts to try without doing that. Anyway, upgrading the firmware is the first step to take if you experience any communication problem with the board. As far as we know this installer only works on Windows, so you would need a Windows system to upgrade the STLINK firmware.

# Uploading the SLab firmware

After the drivers installation, you shall see that the board is seen from the PC at least as a mass storage device, with name **"NODE_F303RE"** in the case of the F303RE Board, and as a COM serial link. If you are using another board the name could be different.

In a Windows system you can see the COM port used in the device manager:

![win com](images\Board_01\win_com.png)

And also the mass storage device:

![win com](images\Board_01\win_storage.png)

You can also see both in the STM32 STLink properties under "Devices and Printers" :

![stlink properties](images\Board_01\stlink.png)

In a Linux machine, the board also is shown as a "NODE_F303RE" external storage unit and, in the machines tested, the serial is seen as **"/dev/ttyACM0"**.

Now it is time to upload the firmware on the board. For official boards, the firmware file has a name pattern:  
<BR>
    <center><B>SLAB-Board Name-X.Y.bin</B></center>
<BR>    
Where "Board Name" is the name of the board and X.Y are the major a minor versions of the firmware. As an example the filename of version 2.0 of the firware for the F303 board is:
    
<BR>
    <center><B>SLab-Nucleo64-F303RE-2.0.bin</B></center>
<BR>

The firmware file can be located inside the **SLab/Boards/BNAME/Firmware/Bin** folder on the standard **SLab** distribution, where **BNAME** is the name of the board. 

To program the firmware just drag and drop the file over the **NODE_F303RE** mass storage disk as if you where copying a file. You will see the board LED next to the USB connector blink during some time and is kept in still in green afterwards.

If all goes ok, you have successfully written the firmware. This procedure has been tested both in Windows and Linux machines.

## Testing the communication

If you want to verify that the firmware is loaded and that the communication link is in operation, you can use a terminal emulator like **Putty** for that. This is not mandatory so you can skip to next section if you want.

If you connect to the board using its COM port at **38400 baud, 8 bits, no parity**, you will see that every time you **reset** the board by hitting its **black button** the board identifies itself with its name like in the following image:

![terminal](images\Board_01\terminal.png)

The text could be different in your case if you are using a newer firmware.

## Serial access in Linux

In a Linux machine you can use the screen program to access the serial port. Note, that you usually need to have root privileges to access the serial port. The following command has been tested to work to open the serial channel on a Linux Ubuntu machine:

>`> sudo screen /dev/ttyACM0 38400`

After entering the communication program, **reset** the board as in the Windows example and see if it responds with its firmware identification string.

Having to be **root** to use the serial port is a hassle. Moreover, it is a security issue because we will need to run **Python** as **root** and the use of **sudo** usually can conflict with your normal environment variables.

If you are in a Linux machine, it is recommended to be able to access to the serial ports without using Linux. One way to do that is removing the **modem manager** package and adding your user to the **dialup** group. The following two commands, executed from **bash** take care of that. Note that in the second case you need to replace **username** with your own user name.

>`> sudo apt-get autoremove modemmanager`

>`> sudo adduser username dialout`

You will probably need to **reboot** for the changes to take effect, but you should now be able to use the serial ports without **sudo**.


After you are sure that all the hardware operates as expected and you can confidently go to the next software section.

## Connection details

This section gives some details about how the **SLab** connects with the board. You can skip it if you don't want or need the details.

You can test the **SLab** conectivity from a Python console.

You can access **SLab** from the **SLab_Console** executable on the **SLab/Console** folder or by running **Python** from the **Slab/Code** folder. This last method, however, has the inconvenient that doesn't instruct **SLab** to use the proper **calibration** folder. Calibration was more important in the first **SLab** releases. Currently you can work totally fine on an uncalibrated board if you don't mind some small measurement errors.

Either way open the console.

In the case of **Linux**, it is possible that you need to access Python as **super user** by using **sudo** in order to be able to connect with the board. To do that you can use a line like:

>`sudo python`

You should not need to do that if you have configured your user in the **dialup** group as instructed previously.

After calling the Python interpreter you should get to the Python prompt. Now write the following line and press enter. Remember that the "\>\>\>" is written by Python, not by you.

>`>>> import slab`

Python should respond something like that:

>`SLAB Module
Version 1.0 (22/2/2017)
Running interactively`

The above import is not needed if you access the console by executing **SLab_Console** as this script automatically imports the **SLab** module.

If it gives any error, check that you have really executed the Python module executable from the proper location. 

You can execute the following **code cell** to do the same from this notebook:

## Connecting with the board

Before connnecting with the board, we need to import the **SLab** module.

In [ ]:
import slab

The **slab.py** module interacts with the board firmware using a serial **COM** port at **38400 baud** speed. For the connection to work, the drivers of the board shall be installed so it is detected by the system.

In order to obtain a successful connection you usually need to know the name of the port. In windows it is usually **"COMn"** where n is a number and you can usually request its number on Devices and Printers on the windows menu or in the device administration control panel. 
In Linux, the name can be something like **"/dev/ttyACM0"**.
As you have seen on a previous figure in this document, the name is **COM52** in my windows machine. In other operating systems knowing the port name will need a different method. You can refer to the Python [pyserial](https://pythonhosted.org/pyserial/) documentation or the operating system information to obtain more details.

Once you have the port name of the hardware board, input the following command on the Python console, changing **'COM52'** with whatever name your serial port has:

>`>>> slab.connect('COM52')`

In Linux it could be:

>`>>> slab.connect('/dev/ttyACM0')`

Python should respond with something like:

>`Getting board data
Connected to Nucleo64-F303RE SLab v1.0
No ADC calibration data found
No DAC calibration data found`

If the port name is not a proper port it will respond something like the following two lines following with some more exception information:

>`** SLab exception
** Cannot open connection. Check port`

If the port name exists on the system but it is not the board with the proper firmware loaded it will respond something like:

>`** Slab exception
** Board not responding. Check port and Firmware`

Or like:

>`** Slab exception
** Bad magic from board. Check Firmware`

In the first case there is not response from the com port after one second and in the second case the response from the board is wrong. The Slab board firmware contains a 4 byte magic code that guarantees that you only get a proper connection if you select a port that links to a board with the proper Slab firmware.

As you can see, the Python SLab module responds with an exception every time something goes wrong. That automatically aborts any running program. This is typical way to handle errors in Python.

If you are not able not obtain the proper name of the COM port used by the board, or you don't bother to find it, you can request the Slab module to autodetect the port. In order to do that, just request the connection without providing any port name:

>`>>>slab.connect()`

That should usually work on Windows, probably on Linux and we don't know on a Mac. If the board can be detected you will see something like:

>`Board detected at port COM52
Getting board data
Connected to Nucleo64-F303RE SLab v1.0
No ADC calibration data found
No DAC calibration data found`

You can use the following code to connect with the board:

In [ ]:
# Connect with the board using autodetection
slab.connect()

You can take note of the COM port nave for successive connections but the Python SLab module will also remember this port for you. The last successfully COM port opened is stored in a **“Last_COM.dat”** file. Check that this file is generated in the **Code** folder.

Next time the board tries to auto detect the port, it will first try to use the last valid COM port like in the next example:

>`>>> slab.connect()`

>`Trying last valid port COM52
Board detected
Getting board data
Connected to Nucleo64-F303RE SLab v1.0
No ADC calibration data found
No DAC calibration data found`

Using auto detect is always slower (although it is quite fast) than providing the proper port name. But thanks to the **“Last_COM.dat”** file the full auto detect algorithm will only need to be carried out once. Moreover, the “Last_COM.dat” file is also stored for manually selected ports, so, except in rare cases when autodetect fails for the first time, you can always use the connect command without arguments.

The auto detect algorithm works by requesting a magic code from all connected devices. As this is an active search, it can affect the normal operation of the devices in the system. If you detect any problem, refrain from using auto detect and manually select the port at least until the "Last_COM.dat" file is generated.

If a board cannot be detected automatically you will get an exception:

>`** SLab exception
** COM Autodetect Fail`

It usually means that there is a problem with the drivers, the STLINK firmware or the board SLab firmware. Off course it can also mean that the board is not connecte to the computer. You can check using a terminal emulator on the board port at 38400 baud that the boards reports its name when you reset it, but if this works, the SLab connection will usually work too.

It can also mean that you are already connected to the board with another Python interpreter. Remeber that you can only have one active connection between the board and a Python interpreter.

If the connection is established, you can disconnect from the board afterwards with the command:

>`>>> slab.disconnect()`

Python will respond with something like:

>`Disconnected from the Board`

You can test this using the code cell below:

In [ ]:
# Disconnect from the board
slab.disconnect()

As you have seen, and usual in python, we have used the slab module name for all the commands. All commands will have the structure:

>`var = slab.command_name(Arguments)`

Where **command_name** is the name of the command and **Arguments** is an optional argument list for the command. Some commands return values. In those cases you can assign the result to a variable **var** or, in some cases, a **list** of variables. For commands that do not return any value, or for the cases that the command returns data you don't need, you can omit the **"var ="** part of the call. As this is standard in Python refer to the Python literature for more information about this language.

It is not recommended to remove the slab reference by using:

>`>>> from slab import *`

It is bad Python practice and can break the operation of the program in unexpected ways. If you want to reduce the number of characters you are writing you'd better use an alias name for the module:

>`>>> import slab as sl`  
>`>>> sl.connect()`  
>`>>> sl.disconnect()`

Before leaving Python, both in interactive mode or using a script, it is recommended to disconnect from the board issuing:

>`>>> slab.disconnect()`

But probably Python will do that for you if you forget it.

## References

**STM32 Nucleo Page**  
Main page on the ST website about the STM32 Nucleo demonstration boards.  
http://www.st.com/en/evaluation-tools/stm32-mcu-nucleo.html

**MBED Developer Page**  
MBED developer page. This page gives access to the MBED compiler.  
https://developer.mbed.org/

**Python page:**  
Main page of the Python Software Foundation.  
https://www.python.org/

**Anaconda page:**  
Page where the Anaconda Python distribution can be downloaded.
https://www.continuum.io/downloads

**SLab Python References**  
Those are the [reference documents](http://localhost:8888/tree/Reference) for the SLab Python modules. They describe the commands that can be carried out after importing each module. 


## Document license

Copyright  ©  Vicente Jiménez (2018-2019)
This work is licensed under a Creative Common Attribution-ShareAlike 4.0 International license.  
This license is available at http://creativecommons.org/licenses/by-sa/4.0/

<img  src="images/cc_sa.png" width="200">